In [ ]:
# ==================================================================================================================
# ===========================================> Instalación librerías <==============================================
# ==================================================================================================================

#%pip install pandas    # NO HACE FALTA INSTALAR
#%pip install numpy     # NO HACE FALTA INSTALAR
#%pip install pickle    # NO HACE FALTA INSTALAR
#%pip install sklearn   # NO HACE FALTA INSTALAR

#%pip install scikit-learn          # DESCOMENTAR LA PRIMERA VEZ
#%pip install category_encoders     # DESCOMENTAR LA PRIMERA VEZ
#%pip install lightgbm              # DESCOMENTAR LA PRIMERA VEZ
#%pip install matplotlib            # DESCOMENTAR LA PRIMERA VEZ
#%pip install seaborn               # DESCOMENTAR LA PRIMERA VEZ



In [ ]:
# ==================================================================================================================
# ===========================================> Carga librerías <====================================================
# ==================================================================================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ------------------------------------------------------------------------------
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import accuracy_score, confusion_matrix, auc, plot_roc_curve, roc_curve, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, auc, RocCurveDisplay , roc_curve, classification_report

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [26]:
# ==================================================================================================================
# ===========================================> Carga Datos <========================================================
# ==================================================================================================================


#datos = pd.read_csv('../data/train.csv', low_memory=False)
datos = pd.read_csv('../data/train_dev.csv', low_memory=False)         # 20.000 filas del fichero train para trabajar en desarrollo

# =====================> EDA

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración número de columnas <=====================================
# ==================================================================================================================

len(datos.columns)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col numericas <=====================
# ==================================================================================================================

datos.describe()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col objectos <=====================
# ==================================================================================================================

datos.describe(include=object)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración 20 primeras filas <======================================
# ==================================================================================================================

datos.head(20)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración tipos <==================================================
# ==================================================================================================================

datos.info()

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración nulos <==================================================
# ==================================================================================================================

datos.isna().sum().sort_values(ascending=False)

In [ ]:
# ==================================================================================================================
# ===========================================> Exploración valores y estadísticas <=================================
# ==================================================================================================================

stats_1 = []
# Unique Values         col 
# Unique Values         datos[col].nunique() 
# Unique Values %       (datos[col].nunique() / datos[col].count()) * 100
# Missing Values %      datos[col].isnull().sum() * 100 / datos.shape[0]
# Biggest Category %    datos[col].value_counts(normalize=True, dropna=False).values[0] * 100
# Type                  datos[col].dtype

for col in datos.columns:
    stats_1.append((col, datos[col].nunique(), (datos[col].nunique() / datos[col].count()) * 100, datos[col].isnull().sum() * 100 / datos.shape[0], datos[col].value_counts(normalize=True, dropna=False).values[0] * 100, datos[col].dtype))
    
stats = pd.DataFrame(stats_1, columns=['Feature', 'Unique Values', 'Unique Values %', 'Missing Values %', 'Biggest Category %', 'Type'])
stats.sort_values('Unique Values %', ascending=False)

In [27]:
# ==================================================================================================================
# ===========================================> Eliminar valores nulos <=============================================
# ==================================================================================================================

drop_cols_min_nulls = 0.7
drop_cols_nulls = []

for col in datos.columns:
    if datos[col].isna().sum() / len(datos) >= drop_cols_min_nulls:
        drop_cols_nulls.append(col)

datos.drop(columns=drop_cols_nulls, inplace=True)

drop_cols_nulls

['DefaultBrowsersIdentifier',
 'PuaMode',
 'Census_ProcessorClass',
 'Census_InternalBatteryType',
 'Census_IsFlightingInternal']

In [28]:
# ==================================================================================================================
# ===========================================> Eliminar valores mal balanceados <===================================
# ==================================================================================================================

drop_cols_min_big_cat = 70
drop_cols_big_cat = []

for col in datos.columns:
    if datos[col].value_counts(normalize=True, dropna=False).values[0] * 100 >= drop_cols_min_big_cat:
        drop_cols_big_cat.append(col) 

datos.drop(columns=drop_cols_big_cat, inplace=True)

drop_cols_big_cat

['ProductName',
 'IsBeta',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'HasTpm',
 'Platform',
 'Processor',
 'OsVer',
 'IsProtected',
 'AutoSampleOptIn',
 'SMode',
 'Firewall',
 'UacLuaenable',
 'Census_DeviceFamily',
 'Census_ProcessorManufacturerIdentifier',
 'Census_HasOpticalDiskDrive',
 'Census_OSArchitecture',
 'Census_IsPortableOperatingSystem',
 'Census_GenuineStateName',
 'Census_IsFlightsDisabled',
 'Census_FlightRing',
 'Census_IsVirtualDevice',
 'Census_IsTouchEnabled',
 'Census_IsPenCapable',
 'Census_IsAlwaysOnAlwaysConnectedCapable']

In [ ]:
# ==================================================================================================================
# ===========================================> Eliminar valores identificadores <===================================
# ==================================================================================================================

drop_cols_id = []

datos.drop(columns=drop_cols_id, inplace=True)

drop_cols_id

In [29]:
# ==================================================================================================================
# ===========================================> Separar datos por tipos categoricas <================================
# ==================================================================================================================

cat_cols = datos.select_dtypes(include=['object', 'category']).columns.to_list()

cat_cols

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_PrimaryDiskTypeName',
 'Census_ChassisTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSVersion',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_ActivationChannel']

In [30]:
# ==================================================================================================================
# ===========================================> Separar datos por tipos numericas <==================================
# ==================================================================================================================

numeric_cols = datos.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('HasDetections') # Posibles labels HasDetections or IsProtected

numeric_cols

['AVProductStatesIdentifier',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsBuild',
 'OsSuite',
 'IeVerIdentifier',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_TotalPhysicalRAM',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_ThresholdOptIn',
 'Census_FirmwareManufacturerIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Census_IsSecureBootEnabled',
 'Census_IsWIMBootEnabled',
 'Wdft_IsGamer',
 'Wdft_RegionIdentifier']

In [31]:
# ==================================================================================================================
# ===========================================> Separar datos numericas a categoricas <==============================
# ==================================================================================================================

numeric_cols_cat_min = 1
numeric_cols_cat = []

for col in numeric_cols:
    if (datos[col].nunique() / datos[col].count()) * 100 >= numeric_cols_cat_min:
        numeric_cols_cat.append(col) 

for col in numeric_cols_cat:
    cat_cols.append(col)

for col in numeric_cols_cat:
    datos[col] = datos[col].astype("category")

for col in numeric_cols_cat:
    numeric_cols.remove(col)
    
numeric_cols_cat

['AVProductStatesIdentifier',
 'CountryIdentifier',
 'CityIdentifier',
 'GeoNameIdentifier',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalBatteryNumberOfCharges',
 'Census_FirmwareVersionIdentifier']

In [32]:
# ==================================================================================================================
# ===========================================> Cambiar tipo numericas a categoricas <===============================
# ==================================================================================================================

for col in numeric_cols_cat:
    datos[col] = datos[col].astype("object")


In [33]:
# ==================================================================================================================
# ===========================================> Imputar nulos para tipos categoricas <===============================
# ==================================================================================================================

imp_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
datos[cat_cols] = imp_cat.fit_transform(datos[cat_cols])

datos.isnull().sum()

MachineIdentifier                                        0
EngineVersion                                            0
AppVersion                                               0
AvSigVersion                                             0
AVProductStatesIdentifier                                0
CountryIdentifier                                        0
CityIdentifier                                           0
OrganizationIdentifier                                6164
GeoNameIdentifier                                        0
LocaleEnglishNameIdentifier                              0
OsBuild                                                  0
OsSuite                                                  0
OsPlatformSubRelease                                     0
OsBuildLab                                               0
SkuEdition                                               0
IeVerIdentifier                                        128
SmartScreen                                             

In [34]:
# ==================================================================================================================
# ===========================================> Imputar nulos para tipos numericas <=================================
# ==================================================================================================================

imp_num = SimpleImputer(missing_values=np.nan, strategy='mean')
datos[numeric_cols] = imp_num.fit_transform(datos[numeric_cols])

datos.isnull().sum()

MachineIdentifier                                    0
EngineVersion                                        0
AppVersion                                           0
AvSigVersion                                         0
AVProductStatesIdentifier                            0
CountryIdentifier                                    0
CityIdentifier                                       0
OrganizationIdentifier                               0
GeoNameIdentifier                                    0
LocaleEnglishNameIdentifier                          0
OsBuild                                              0
OsSuite                                              0
OsPlatformSubRelease                                 0
OsBuildLab                                           0
SkuEdition                                           0
IeVerIdentifier                                      0
SmartScreen                                          0
Census_MDC2FormFactor                                0
Census_OEM

In [35]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 3 partes <=============================
# ==================================================================================================================

mask_cols_3 = []

for col in datos.columns:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 2).all():
        mask_cols_3.append(c)

for col in mask_cols_3:
    datos[[col + "_1", col + "_2", col + "_3"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_3:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")

datos.drop(columns=mask_cols_3, inplace=True)

mask_cols_3

[]

In [36]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 4 partes <=============================
# ==================================================================================================================

mask_cols_4 = []

for col in datos.columns:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 3).all():
        mask_cols_4.append(col)

for col in mask_cols_4:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_4:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")

datos.drop(columns=mask_cols_4, inplace=True)

mask_cols_4

['EngineVersion', 'AppVersion', 'AvSigVersion', 'Census_OSVersion']

In [37]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 5 partes <=============================
# ==================================================================================================================

mask_cols_5 = []

for col in datos.columns:
    if datos[col].notnull().all() and datos[col].astype(str).apply(lambda x: x.count('.') == 4).all():
        mask_cols_5.append(col)

for col in mask_cols_5:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4", col + "_5"]] = datos[col].str.split(".", expand=True)

for col in mask_cols_5:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")
    cat_cols.append(col + "_5")

datos.drop(columns=mask_cols_5, inplace=True)

mask_cols_5

['OsBuildLab']

In [38]:
# ==================================================================================================================
# ===========================================> Procesar mascaras y versiones 6 partes <=============================
# ==================================================================================================================

mask_cols_6 = []

for c in datos.columns:
    if datos[c].notnull().all() and datos[c].astype(str).apply(lambda x: x.count('.') == 5).all():
        mask_cols_6.append(c)

for c in mask_cols_6:
    datos[[col + "_1", col + "_2", col + "_3", col + "_4", col + "_5", col + "_6"]] = datos[c].str.split(".", expand=True)

for col in mask_cols_6:
    cat_cols.remove(col)
    cat_cols.append(col + "_1")
    cat_cols.append(col + "_2")
    cat_cols.append(col + "_3")
    cat_cols.append(col + "_4")
    cat_cols.append(col + "_5")
    cat_cols.append(col + "_6")

datos.drop(columns=mask_cols_6, inplace=True)

mask_cols_6

[]

In [40]:
cat_cols

['MachineIdentifier',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'SmartScreen',
 'Census_MDC2FormFactor',
 'Census_PrimaryDiskTypeName',
 'Census_ChassisTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSVersion',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_ActivationChannel',
 'AVProductStatesIdentifier',
 'CountryIdentifier',
 'CityIdentifier',
 'GeoNameIdentifier',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalBatteryNumberOfCharges',
 'Census_FirmwareVersionIdentifier']

In [39]:
# ==================================================================================================================
# ===========================================> Ordenar variables <==================================================
# ==================================================================================================================

ordered_columns = cat_cols+numeric_cols
datos = datos[ordered_columns+['HasDetections']]

KeyError: "['EngineVersion', 'AppVersion', 'AvSigVersion', 'OsBuildLab', 'Census_OSVersion'] not in index"

In [ ]:
# ==================================================================================================================
# ===========================================> Codificar variables <================================================
# ==================================================================================================================

preprocessor = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                    remainder='passthrough')

In [ ]:
# ==================================================================================================================
# ===========================================> Train Test Split <===================================================
# ==================================================================================================================

X_train, X_test, y_train, y_test = train_test_split(
                                        datos.drop(columns = 'HasDetections')[ordered_columns],
                                        datos['HasDetections'],
                                        random_state = 123)

X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep  = preprocessor.transform(X_test)

encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names_out(cat_cols)
labels = np.concatenate([encoded_cat, numeric_cols])

X_train_prep = pd.DataFrame(X_train_prep.toarray(), columns=labels)
X_test_prep  = pd.DataFrame(X_test_prep.toarray(), columns=labels)
X_train_prep.info()

In [ ]:
# ==================================================================================================================
# ===========================================> Modelo <=============================================================
# ==================================================================================================================

modelo = DecisionTreeClassifier(criterion= "entropy", max_depth=5, min_samples_leaf=3, random_state = 123)

modelo.fit(X_train_prep, y_train)

fig, ax = plt.subplots(figsize=(16, 6))

plot = plot_tree(
            decision_tree = modelo,
            feature_names = labels.tolist(),
            class_names   = 'HasDetections',
            filled        = True,
            impurity      = False,
            fontsize      = 7,
            ax            = ax)

In [ ]:
predicciones = modelo.predict(X = X_test_prep)
pred_proba = modelo.predict_proba(X = X_test_prep)

In [ ]:
importancia_predictores = pd.DataFrame({
    "predictor": labels.tolist(),
    "importancia": modelo.feature_importances_
})

fig, ax = plt.subplots(figsize=(16, 6))

importancia_predictores.set_index("predictor").sort_values("importancia", ascending=True)[10:].plot(kind="barh", ax=ax)


In [ ]:
predicciones = modelo.predict(X = X_test_prep,)
pred_proba = modelo.predict_proba(X = X_test_prep)

data = confusion_matrix(y_test, predicciones)

print("Matriz de confusión")
plt.figure(figsize = (6,5))
sns.set(font_scale=1.4) #for label size
sns.heatmap(data, cmap="Blues", annot=True, annot_kws={"size": 12})


In [ ]:
print(f'Accuracy: {round(100*accuracy_score(y_test, predicciones),1)}%')

In [ ]:
print(classification_report(y_test, predicciones, digits=3, zero_division=True))

In [ ]:
fpr, tpr, _ = roc_curve(y_test, pred_proba[:,1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize = (6,5))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar Unique Tipos Cat <==========================================
# ==================================================================================================================

#typed_min_unique = 10
#typed_cols = []

#for c in datos.columns:
#    if ((datos[c].nunique() / datos[c].count()) * 100) <= typed_min_unique:
#        typed_cols.append(c)
        
#for c in typed_cols:
#    datos[c] = datos[c].astype("category")

#print(typed_cols)

In [ ]:
# ==================================================================================================================
# ===========================================> Procesar Object Tipos Cat <==========================================
# ==================================================================================================================

#typed_cols = []
#cat_cols = []

#for c in datos.columns:
#    if datos[c].dtype == "object":
#        typed_cols.append(c)
#        cat_cols.append(c)
        
#for c in typed_cols:
#   datos[c] = datos[c].astype("category")

#datos["HasDetections"] = datos["HasDetections"].astype(int)

#print(typed_cols)

In [ ]:
#from sklearn.compose import ColumnTransformer


#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score


# Dividir los datos en características (X) y etiquetas (y)
#X = datostos.drop('HasDetections', axis=1)
#y = datos['HasDetections']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo (por ejemplo, RandomForestClassifier)
#model = RandomForestClassifier()
#model.fit(X_train, y_train)


# Realizar predicciones en el conjunto de prueba
#y_pred = model.predict(X_test)

# Calcular la precisión del modelo
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Precisión del modelo: {accuracy}")



cat_cols = datos.select_dtypes(include=['object', 'category']).columns.to_list()
numeric_cols = datos.select_dtypes(include=['float64', 'int64']).columns.to_list()

preprocessor = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                    remainder='passthrough')

X_train, X_test, y_train, y_test = train_test_split(
                                        datos.drop(columns = 'HasDetections'),
                                        datos['HasDetections'],
                                        random_state = 123)

X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep  = preprocessor.transform(X_test)

encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names_out(cat_cols)
labels = np.concatenate([cat_cols, numeric_cols])

# Conversión a dataframe
X_train_prep = pd.DataFrame(X_train_prep.toarray(), columns=labels)
X_test_prep  = pd.DataFrame(X_test_prep.toarray(), columns=labels)
X_train_prep.info()

modelo = DecisionTreeClassifier(max_depth=3, random_state = 123)

modelo.fit(X_train_prep, y_train)

# Estructura del árbol creado
# ------------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(16, 6))

print(f"Profundidad del árbol: {modelo.get_depth()}")
print(f"Número de nodos terminales: {modelo.get_n_leaves()}")

#plot = plot_tree(
#            decision_tree = modelo,
#            feature_names = labels.tolist(),
#            class_names   = 'HasDetections',
#            filled        = True,
#            impurity      = False,
#            fontsize      = 7,
#            ax            = ax)